# Define Library

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid
from sklearn.metrics import roc_auc_score
from datetime import datetime, timedelta
import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io
path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)

# Function

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import auc, roc_curve
from itertools import product
import warnings
warnings.filterwarnings('ignore')

class GiniDashboardGenerator:
    """Generate Gini/AUC metrics and segmentation tables for Power BI dashboard"""
    
    def __init__(self, df, score_column, target_column):
        """
        Initialize with configurable score and target columns
        
        Args:
            df: Input dataframe
            score_column: Name of the score column (e.g., 'Alpha_cic_sil_score')
            target_column: Name of the target column (e.g., 'deffpd0', 'deffpd10', etc.)
        """
        self.df = df.copy()
        self.score_column = score_column
        self.target_column = target_column
        self.prepare_data()
        
    def prepare_data(self):
        """Prepare data with disbursement month and week columns"""
        self.df['disbursementdate'] = pd.to_datetime(self.df['disbursementdate'])
        self.df['disbursement_month'] = self.df['disbursementdate'].dt.to_period('M')
        self.df['disbursement_week'] = self.df['disbursementdate'].dt.to_period('W')
        
        # Standardize column names for internal processing
        self.df['score'] = self.df[self.score_column]
        self.df['Bad_Rate'] = self.df[self.target_column]
        
        # Add metadata to track original column names
        self.metadata = {
            'score_column': self.score_column,
            'target_column': self.target_column
        }
        
    def calculate_gini_auc(self, y_true, y_score):
        """Calculate AUC and Gini coefficient"""
        if len(np.unique(y_true)) < 2 or len(y_true) == 0:
            return np.nan, np.nan
        
        try:
            fpr, tpr, _ = roc_curve(y_true, y_score)
            auc_score = auc(fpr, tpr)
            gini = 2 * auc_score - 1
            return auc_score, gini
        except:
            return np.nan, np.nan
    
    def generate_segments(self):
        """Generate all segment combinations"""
        segments = []
        segment_id = 1
        
        # Get unique values for dimensions
        model_versions = self.df.groupby(['modelDisplayName', 'modelVersionId', 'trenchCategory']).size().reset_index()[
            ['modelDisplayName', 'modelVersionId', 'trenchCategory']].drop_duplicates()
        
        new_loan_types = self.df['new_loan_type'].unique().tolist()
        loan_product_types = self.df['loan_product_type'].unique().tolist()
        
        # Combination 1: Overall for each modelDisplayName+modelVersionId+trenchCategory
        # Include ALL trench combined for each modelDisplayName+modelVersionId
        for _, row in model_versions.iterrows():
            model_name = row['modelDisplayName']
            model_version = row['modelVersionId']
            trench = row['trenchCategory']
            
            segments.append({
                'segment_id': segment_id,
                'segment_name': f"{model_name}_{model_version}_{trench}",
                'modelDisplayName': model_name,
                'modelVersionId': model_version,
                'trenchCategory': trench,
                'new_loan_type': 'All',
                'loan_product_type': 'All',
                'segment_type': 'Trench_Level'
            })
            segment_id += 1
        
        # Add "All" trench combining Trench 1, 2, 3
        for model_name in self.df['modelDisplayName'].unique():
            for model_version in self.df[self.df['modelDisplayName'] == model_name]['modelVersionId'].unique():
                segments.append({
                    'segment_id': segment_id,
                    'segment_name': f"{model_name}_{model_version}_All",
                    'modelDisplayName': model_name,
                    'modelVersionId': model_version,
                    'trenchCategory': 'All',
                    'new_loan_type': 'All',
                    'loan_product_type': 'All',
                    'segment_type': 'Model_Level'
                })
                segment_id += 1
        
        # Combination 2 & 3: With new_loan_type and loan_product_type
        for _, row in model_versions.iterrows():
            model_name = row['modelDisplayName']
            model_version = row['modelVersionId']
            trench = row['trenchCategory']
            
            # Combination 3: Each combination of new_loan_type and loan_product_type
            for loan_type in new_loan_types:
                segments.append({
                    'segment_id': segment_id,
                    'segment_name': f"{model_name}_{model_version}_{trench}_{loan_type}",
                    'modelDisplayName': model_name,
                    'modelVersionId': model_version,
                    'trenchCategory': trench,
                    'new_loan_type': loan_type,
                    'loan_product_type': 'All',
                    'segment_type': 'Loan_Type_Level'
                })
                segment_id += 1
            
            for product_type in loan_product_types:
                segments.append({
                    'segment_id': segment_id,
                    'segment_name': f"{model_name}_{model_version}_{trench}_{product_type}",
                    'modelDisplayName': model_name,
                    'modelVersionId': model_version,
                    'trenchCategory': trench,
                    'new_loan_type': 'All',
                    'loan_product_type': product_type,
                    'segment_type': 'Product_Type_Level'
                })
                segment_id += 1
            
            # Both new_loan_type and loan_product_type
            for loan_type in new_loan_types:
                for product_type in loan_product_types:
                    segments.append({
                        'segment_id': segment_id,
                        'segment_name': f"{model_name}_{model_version}_{trench}_{loan_type}_{product_type}",
                        'modelDisplayName': model_name,
                        'modelVersionId': model_version,
                        'trenchCategory': trench,
                        'new_loan_type': loan_type,
                        'loan_product_type': product_type,
                        'segment_type': 'Full_Level'
                    })
                    segment_id += 1
        
        self.dim_segment = pd.DataFrame(segments)
        return self.dim_segment
    
    def filter_data(self, model_name, model_version, trench, loan_type, product_type):
        """Filter dataframe based on segment criteria"""
        df_filtered = self.df[
            (self.df['modelDisplayName'] == model_name) &
            (self.df['modelVersionId'] == model_version)
        ]
        
        if trench != 'All':
            df_filtered = df_filtered[df_filtered['trenchCategory'] == trench]
        else:
            # Combine all trenches
            trenches = ['Trench 1', 'Trench 2', 'Trench 3']
            df_filtered = df_filtered[df_filtered['trenchCategory'].isin(trenches)]
        
        if loan_type != 'All':
            df_filtered = df_filtered[df_filtered['new_loan_type'] == loan_type]
        
        if product_type != 'All':
            df_filtered = df_filtered[df_filtered['loan_product_type'] == product_type]
        
        return df_filtered
    
    def calculate_score_deciles_and_cumulative(self, df_data, segment_id, time_period, time_type):
        """Calculate decile-based metrics and cumulative statistics for DAX calculations"""
        if len(df_data) == 0:
            return []
        
        decile_records = []
        decile_id = 1
        
        # Sort by score descending to create deciles (rank 1 = highest score)
        df_sorted = df_data.sort_values('score', ascending=False).reset_index(drop=True)
        
        # Create deciles
        df_sorted['decile'] = pd.qcut(df_sorted['score'].rank(method='first'), 
                                       q=10, labels=False, duplicates='drop') + 1
        df_sorted['decile'] = 10 - df_sorted['decile'] + 1  # Reverse so 1 = lowest score, 10 = highest
        
        total_records = len(df_sorted)
        total_bads = df_sorted['Bad_Rate'].sum()
        total_goods = total_records - total_bads
        
        cum_records = 0
        cum_bads = 0
        cum_goods = 0
        
        # Calculate metrics per decile
        for decile in sorted(df_sorted['decile'].unique()):
            df_decile = df_sorted[df_sorted['decile'] == decile]
            decile_records_count = len(df_decile)
            decile_bads = df_decile['Bad_Rate'].sum()
            decile_goods = decile_records_count - decile_bads
            
            cum_records += decile_records_count
            cum_bads += decile_bads
            cum_goods += decile_goods
            
            decile_records.append({
                'segment_id': segment_id,
                'time_period': time_period,
                'time_type': time_type,
                'decile': decile,
                'score_min': df_decile['score'].min(),
                'score_max': df_decile['score'].max(),
                'score_avg': df_decile['score'].mean(),
                'records_in_decile': decile_records_count,
                'bads_in_decile': int(decile_bads),
                'goods_in_decile': decile_goods,
                'pct_records': (decile_records_count / total_records * 100) if total_records > 0 else 0,
                'pct_bads': (decile_bads / total_bads * 100) if total_bads > 0 else 0,
                'pct_goods': (decile_goods / total_goods * 100) if total_goods > 0 else 0,
                'cum_records': cum_records,
                'cum_bads': int(cum_bads),
                'cum_goods': cum_goods,
                'cum_pct_records': (cum_records / total_records * 100) if total_records > 0 else 0,
                'cum_pct_bads': (cum_bads / total_bads * 100) if total_bads > 0 else 0,
                'cum_pct_goods': (cum_goods / total_goods * 100) if total_goods > 0 else 0,
                'bad_rate_decile': (decile_bads / decile_records_count * 100) if decile_records_count > 0 else 0,
                'total_records': total_records,
                'total_bads': total_bads,
                'total_goods': total_goods
            })
            decile_id += 1
        
        return decile_records
    
    def calculate_metrics(self):
        """Calculate AUC and Gini for all segments and time periods"""
        facts = []
        deciles = []
        fact_id = 1
        
        for idx, segment in self.dim_segment.iterrows():
            df_segment = self.filter_data(
                segment['modelDisplayName'],
                segment['modelVersionId'],
                segment['trenchCategory'],
                segment['new_loan_type'],
                segment['loan_product_type']
            )
            
            # Overall metrics
            if len(df_segment) > 0:
                auc_score, gini = self.calculate_gini_auc(
                    df_segment['Bad_Rate'].values,
                    df_segment['score'].values
                )
                
                facts.append({
                    'fact_id': fact_id,
                    'segment_id': segment['segment_id'],
                    'segment_rank': idx + 1,
                    'time_period': 'Overall',
                    'time_type': 'Overall',
                    'disbursement_month': 'All',
                    'disbursement_week': 'All',
                    'auc': auc_score,
                    'gini': gini,
                    'record_count': len(df_segment),
                    'bad_count': int(df_segment['Bad_Rate'].sum()),
                    'good_count': len(df_segment) - int(df_segment['Bad_Rate'].sum()),
                    'bad_rate_pct': (df_segment['Bad_Rate'].sum() / len(df_segment) * 100) if len(df_segment) > 0 else 0
                })
                fact_id += 1
                
                # Generate decile data for this segment
                decile_data = self.calculate_score_deciles_and_cumulative(
                    df_segment, segment['segment_id'], 'Overall', 'Overall'
                )
                deciles.extend(decile_data)
            
            # Monthly metrics
            for month in df_segment['disbursement_month'].unique():
                df_month = df_segment[df_segment['disbursement_month'] == month]
                
                if len(df_month) > 0:
                    auc_score, gini = self.calculate_gini_auc(
                        df_month['Bad_Rate'].values,
                        df_month['score'].values
                    )
                    
                    facts.append({
                        'fact_id': fact_id,
                        'segment_id': segment['segment_id'],
                        'segment_rank': idx + 1,
                        'time_period': str(month),
                        'time_type': 'Month',
                        'disbursement_month': str(month),
                        'disbursement_week': 'All',
                        'auc': auc_score,
                        'gini': gini,
                        'record_count': len(df_month),
                        'bad_count': int(df_month['Bad_Rate'].sum()),
                        'good_count': len(df_month) - int(df_month['Bad_Rate'].sum()),
                        'bad_rate_pct': (df_month['Bad_Rate'].sum() / len(df_month) * 100) if len(df_month) > 0 else 0
                    })
                    fact_id += 1
                    
                    # Generate decile data for this month
                    decile_data = self.calculate_score_deciles_and_cumulative(
                        df_month, segment['segment_id'], str(month), 'Month'
                    )
                    deciles.extend(decile_data)
            
            # Weekly metrics
            for week in df_segment['disbursement_week'].unique():
                df_week = df_segment[df_segment['disbursement_week'] == week]
                
                if len(df_week) > 0:
                    auc_score, gini = self.calculate_gini_auc(
                        df_week['Bad_Rate'].values,
                        df_week['score'].values
                    )
                    
                    facts.append({
                        'fact_id': fact_id,
                        'segment_id': segment['segment_id'],
                        'segment_rank': idx + 1,
                        'time_period': str(week),
                        'time_type': 'Week',
                        'disbursement_month': 'All',
                        'disbursement_week': str(week),
                        'auc': auc_score,
                        'gini': gini,
                        'record_count': len(df_week),
                        'bad_count': int(df_week['Bad_Rate'].sum()),
                        'good_count': len(df_week) - int(df_week['Bad_Rate'].sum()),
                        'bad_rate_pct': (df_week['Bad_Rate'].sum() / len(df_week) * 100) if len(df_week) > 0 else 0
                    })
                    fact_id += 1
                    
                    # Generate decile data for this week
                    decile_data = self.calculate_score_deciles_and_cumulative(
                        df_week, segment['segment_id'], str(week), 'Week'
                    )
                    deciles.extend(decile_data)
        
        self.fact_metrics = pd.DataFrame(facts)
        self.fact_deciles = pd.DataFrame(deciles)
        return self.fact_metrics
    
    def generate_all(self):
        """Generate all tables"""
        self.generate_segments()
        self.calculate_metrics()
        
        return {
            'fact_metrics': self.fact_metrics,
            'fact_deciles': self.fact_deciles,
            'dim_segment': self.dim_segment,
            'metadata': self.metadata
        }
    
    def save_tables(self, output_path='./', target_suffix=''):
        """
        Save fact and dimension tables to CSV
        
        Args:
            output_path: Path to save files
            target_suffix: Suffix to append to filenames (e.g., '_deffpd0', '_deffpd10')
        """
        suffix = f'_{target_suffix}' if target_suffix else ''
        
        self.fact_metrics.to_csv(f'{output_path}fact_metrics{suffix}.csv', index=False)
        self.fact_deciles.to_csv(f'{output_path}fact_deciles{suffix}.csv', index=False)
        self.dim_segment.to_csv(f'{output_path}dim_segment{suffix}.csv', index=False)
        
        print(f"✓ Fact metrics table saved: {output_path}fact_metrics{suffix}.csv ({len(self.fact_metrics)} rows)")
        print(f"✓ Fact deciles table saved: {output_path}fact_deciles{suffix}.csv ({len(self.fact_deciles)} rows)")
        print(f"✓ Dimension segment table saved: {output_path}dim_segment{suffix}.csv ({len(self.dim_segment)} rows)")
        print(f"  Score Column: {self.metadata['score_column']}, Target Column: {self.metadata['target_column']}")


# # Usage Example with Multiple Targets
# if __name__ == "__main__":
    
#     # Define all target columns and their corresponding score columns
#     targets = [
#         {'target': 'deffpd0', 'score': 'Alpha_cic_sil_score', 'suffix': 'deffpd0'},
#         {'target': 'deffpd10', 'score': 'Alpha_cic_sil_score', 'suffix': 'deffpd10'},
#         {'target': 'deffpd30', 'score': 'Alpha_cic_sil_score', 'suffix': 'deffpd30'},
#     ]
    
#     all_fact_metrics = []
#     all_fact_deciles = []
#     all_dim_segments = []
    
#     # Process each target
#     for config in targets:
#         print(f"\n{'='*60}")
#         print(f"Processing: {config['suffix']}")
#         print(f"{'='*60}")
        
#         # Initialize generator with specific score and target columns
#         generator = GiniDashboardGenerator(
#             df_concat,
#             score_column=config['score'],
#             target_column=config['target']
#         )
        
#         # Generate all tables
#         tables = generator.generate_all()
        
#         # Save individual tables with suffix
#         generator.save_tables(output_path='./', target_suffix=config['suffix'])
        
#         # Add target column info for concatenation
#         tables['fact_metrics']['target_column'] = config['target']
#         tables['fact_deciles']['target_column'] = config['target']
#         tables['dim_segment']['target_column'] = config['target']
        
#         # Collect for concatenation
#         all_fact_metrics.append(tables['fact_metrics'])
#         all_fact_deciles.append(tables['fact_deciles'])
#         all_dim_segments.append(tables['dim_segment'])
        
#         # Display sample
#         print(f"\nSample metrics (first 5 rows):")
#         print(tables['fact_metrics'][['segment_rank', 'time_type', 'gini', 'record_count']].head())
    
#     # Concatenate all results
#     print(f"\n{'='*60}")
#     print("CONCATENATING ALL TARGETS")
#     print(f"{'='*60}")
    
#     combined_fact_metrics = pd.concat(all_fact_metrics, ignore_index=True)
#     combined_fact_deciles = pd.concat(all_fact_deciles, ignore_index=True)
#     combined_dim_segment = pd.concat(all_dim_segments, ignore_index=True).drop_duplicates()
    
#     # Save combined tables
#     combined_fact_metrics.to_csv('./fact_metrics_combined.csv', index=False)
#     combined_fact_deciles.to_csv('./fact_deciles_combined.csv', index=False)
#     combined_dim_segment.to_csv('./dim_segment_combined.csv', index=False)
    
#     print(f"✓ Combined fact metrics: ./fact_metrics_combined.csv ({len(combined_fact_metrics)} rows)")
#     print(f"✓ Combined fact deciles: ./fact_deciles_combined.csv ({len(combined_fact_deciles)} rows)")
#     print(f"✓ Combined dimension segment: ./dim_segment_combined.csv ({len(combined_dim_segment)} rows)")
    
#     # Display combined sample
#     print(f"\n=== COMBINED FACT METRICS SAMPLE ===")
#     print(combined_fact_metrics[['target_column', 'segment_rank', 'time_type', 'gini', 'record_count']].head(10))
    
#     print(f"\n=== DIMENSION SEGMENT (Combined, Unique) ===")
#     print(combined_dim_segment[['segment_id', 'segment_name', 'target_column']].head(10))

## calculate_gini

In [2]:
def calculate_gini(pd_scores, bad_indicators):
    """
    Calculate Gini coefficient from scores and binary indicators
    
    Parameters:
    pd_scores: array-like of scores/probabilities
    bad_indicators: array-like of binary outcomes (0/1)
    
    Returns:
    float: Gini coefficient
    """
    # Convert inputs to numpy arrays and ensure they're numeric
    pd_scores = np.array(pd_scores, dtype=float)
    bad_indicators = np.array(bad_indicators, dtype=int)
    
    # Check for valid input data
    if len(pd_scores) == 0 or len(bad_indicators) == 0:
        return np.nan
    
    # Check if we have both good and bad cases (needed for ROC AUC)
    if len(np.unique(bad_indicators)) < 2:
        return np.nan
    
    # Calculate AUC using sklearn
    try:
        auc = roc_auc_score(bad_indicators, pd_scores)
        # Calculate Gini from AUC
        gini = 2 * auc - 1
        return gini
    except ValueError:
        return np.nan

## calculate_periodic_gini_prod_ver_trench

In [3]:
def calculate_periodic_gini_prod_ver_trench(df, score_column, label_column, namecolumn, 
                                        model_version_column=None, trench_column=None, 
                                        loan_type_column=None, loan_product_type_column=None):
    """
    Calculate periodic Gini coefficients at multiple levels:
    - Overall
    - By model version
    - By all combinations of model version, trench category, loan type, and loan product type
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    namecolumn: name for the bad rate label
    model_version_column: (optional) name of column for model version (e.g., 'modelVersionId')
    trench_column: (optional) name of column for trench category (e.g., 'trenchCategory')
    loan_type_column: (optional) name of loan type column (e.g., 'loan_type')
    loan_product_type_column: (optional) name of loan product type column (e.g., 'loan_product_type')
    """
    # Input validation
    required_columns = ['disbursementdate', score_column, label_column]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
    
    optional_columns = {
        'model_version': model_version_column,
        'trench': trench_column,
        'loan_type': loan_type_column,
        'loan_product_type': loan_product_type_column
    }
    
    for col_name, col in optional_columns.items():
        if col and col not in df.columns:
            raise ValueError(f"{col_name.replace('_', ' ').title()} column '{col}' not found in dataframe")
    
    # Create a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Ensure score and label columns are numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
    # Drop rows with invalid values
    df = df.dropna(subset=[score_column, label_column])
    
    # Define list of datasets to process
    datasets_to_process = [('Overall', df, {})]
    
    # Add model version level segmentation
    if model_version_column:
        for model_version in sorted(df[model_version_column].dropna().unique()):
            model_df = df[df[model_version_column] == model_version]
            metadata = {model_version_column: model_version}
            datasets_to_process.append((f'ModelVersion_{model_version}', model_df, metadata))
    
    # Add all combinations of model_version, trench, loan_type, loan_product_type
    segment_columns = []
    if model_version_column:
        segment_columns.append(('ModelVersion', model_version_column))
    if trench_column:
        segment_columns.append(('Trench', trench_column))
    if loan_type_column:
        segment_columns.append(('LoanType', loan_type_column))
    if loan_product_type_column:
        segment_columns.append(('ProductType', loan_product_type_column))
    
    # Generate all combinations (only if we have multiple segment columns)
    if len(segment_columns) > 1:
        # Create combinations by iterating through all segment dimensions
        def generate_combinations(df, segment_columns, index=0, current_filter=None, current_name=''):
            if current_filter is None:
                current_filter = {}
            
            if index >= len(segment_columns):
                # Apply all filters and create segment
                filtered_df = df
                for col, val in current_filter.items():
                    filtered_df = filtered_df[filtered_df[col] == val]
                
                if len(filtered_df) > 0:
                    yield (current_name.strip('_'), filtered_df, current_filter.copy())
                return
            
            seg_name, seg_col = segment_columns[index]
            for seg_value in sorted(df[seg_col].dropna().unique()):
                new_filter = current_filter.copy()
                new_filter[seg_col] = seg_value
                new_name = current_name + f'{seg_name}_{seg_value}_'
                
                yield from generate_combinations(df, segment_columns, index + 1, new_filter, new_name)
        
        for combo_name, combo_df, combo_metadata in generate_combinations(df, segment_columns):
            datasets_to_process.append((combo_name, combo_df, combo_metadata))
    
    all_results = []
    
    # Process each dataset
    for dataset_name, dataset_df, metadata in datasets_to_process:
        # Calculate weekly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['week'] = dataset_df_copy['disbursementdate'].dt.to_period('W')
        weekly_gini = dataset_df_copy.groupby('week').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 10 else np.nan
        ).reset_index(name='gini')
        weekly_gini['period'] = 'Week'
        weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
        weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
        weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period']]
        
        # Calculate monthly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['month'] = dataset_df_copy['disbursementdate'].dt.to_period('M')
        monthly_gini = dataset_df_copy.groupby('month').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 20 else np.nan
        ).reset_index(name='gini')
        monthly_gini['period'] = 'Month'
        monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
        monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
        monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period']]
        
        # Combine results for this dataset
        gini_results = pd.concat([weekly_gini, monthly_gini], ignore_index=True)
        gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
        
        # Add metadata columns
        gini_results['Model_Name'] = score_column
        gini_results['bad_rate'] = namecolumn
        gini_results['segment_type'] = dataset_name
        
        # Add individual segment components
        if model_version_column and model_version_column in metadata:
            gini_results['model_version'] = metadata[model_version_column]
        else:
            gini_results['model_version'] = None
        
        if trench_column and trench_column in metadata:
            gini_results['trench_category'] = metadata[trench_column]
        else:
            gini_results['trench_category'] = None
        
        if loan_type_column and loan_type_column in metadata:
            gini_results['loan_type'] = metadata[loan_type_column]
        else:
            gini_results['loan_type'] = None
        
        if loan_product_type_column and loan_product_type_column in metadata:
            gini_results['loan_product_type'] = metadata[loan_product_type_column]
        else:
            gini_results['loan_product_type'] = None
        
        gini_results.rename(columns={'gini': f'{score_column}_{namecolumn}_gini'}, inplace=True)
        
        all_results.append(gini_results)
    
    # Combine all results
    final_results = pd.concat(all_results, ignore_index=True)
    
    return final_results


# Usage:
# gini_results = calculate_periodic_gini_prod_ver_trench(
#     df_concat, 
#     'Alpha_cic_sil_score', 
#     'deffpd0', 
#     'FPD0',
#     model_version_column='modelVersionId',
#     trench_column='trenchCategory',
#     loan_type_column='loan_type',
#     loan_product_type_column='loan_product_type'
# )

In [4]:
# def calculate_periodic_gini_prod_ver_trench_md(df, score_column, label_column, namecolumn, 
#                                         model_version_column=None, trench_column=None, 
#                                         loan_type_column=None, loan_product_type_column=None,
#                                         account_id_column=None):
#     """
#     Calculate periodic Gini coefficients at multiple levels:
#     - Overall
#     - By model version
#     - By all combinations of model version, trench category, loan type, and loan product type
    
#     Parameters:
#     df: DataFrame with disbursement dates and score/label columns
#     score_column: name of the score column
#     label_column: name of the label column
#     namecolumn: name for the bad rate label
#     model_version_column: (optional) name of column for model version (e.g., 'modelVersionId')
#     trench_column: (optional) name of column for trench category (e.g., 'trenchCategory')
#     loan_type_column: (optional) name of loan type column (e.g., 'loan_type')
#     loan_product_type_column: (optional) name of loan product type column (e.g., 'loan_product_type')
#     account_id_column: (optional) name of column for distinct account IDs (e.g., 'digitalLoanAccountId')
#     """
#     # Input validation
#     required_columns = ['disbursementdate', score_column, label_column]
#     if not all(col in df.columns for col in required_columns):
#         raise ValueError(f"Missing required columns. Need: {required_columns}")
    
#     optional_columns = {
#         'model_version': model_version_column,
#         'trench': trench_column,
#         'loan_type': loan_type_column,
#         'loan_product_type': loan_product_type_column,
#         'account_id': account_id_column
#     }
    
#     for col_name, col in optional_columns.items():
#         if col and col not in df.columns:
#             raise ValueError(f"{col_name.replace('_', ' ').title()} column '{col}' not found in dataframe")
    
#     # Create a copy to avoid modifying original dataframe
#     df = df.copy()
    
#     # Ensure date is datetime type
#     df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
#     # Ensure score and label columns are numeric
#     df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
#     df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
#     # Drop rows with invalid values
#     df = df.dropna(subset=[score_column, label_column])
    
#     # Define list of datasets to process
#     datasets_to_process = [('Overall', df, {})]
    
#     # Add model version level segmentation
#     if model_version_column:
#         for model_version in sorted(df[model_version_column].dropna().unique()):
#             model_df = df[df[model_version_column] == model_version]
#             metadata = {model_version_column: model_version}
#             datasets_to_process.append((f'ModelVersion_{model_version}', model_df, metadata))
    
#     # Add all combinations of model_version, trench, loan_type, loan_product_type
#     segment_columns = []
#     if model_version_column:
#         segment_columns.append(('ModelVersion', model_version_column))
#     if trench_column:
#         segment_columns.append(('Trench', trench_column))
#     if loan_type_column:
#         segment_columns.append(('LoanType', loan_type_column))
#     if loan_product_type_column:
#         segment_columns.append(('ProductType', loan_product_type_column))
    
#     # Generate all combinations (only if we have multiple segment columns)
#     if len(segment_columns) > 1:
#         # Create combinations by iterating through all segment dimensions
#         def generate_combinations(df, segment_columns, index=0, current_filter=None, current_name=''):
#             if current_filter is None:
#                 current_filter = {}
            
#             if index >= len(segment_columns):
#                 # Apply all filters and create segment
#                 filtered_df = df
#                 for col, val in current_filter.items():
#                     filtered_df = filtered_df[filtered_df[col] == val]
                
#                 if len(filtered_df) > 0:
#                     yield (current_name.strip('_'), filtered_df, current_filter.copy())
#                 return
            
#             seg_name, seg_col = segment_columns[index]
#             for seg_value in sorted(df[seg_col].dropna().unique()):
#                 new_filter = current_filter.copy()
#                 new_filter[seg_col] = seg_value
#                 new_name = current_name + f'{seg_name}_{seg_value}_'
                
#                 yield from generate_combinations(df, segment_columns, index + 1, new_filter, new_name)
        
#         for combo_name, combo_df, combo_metadata in generate_combinations(df, segment_columns):
#             datasets_to_process.append((combo_name, combo_df, combo_metadata))
    
#     all_results = []
    
#     # Process each dataset
#     for dataset_name, dataset_df, metadata in datasets_to_process:
#         # Calculate weekly Gini
#         dataset_df_copy = dataset_df.copy()
#         dataset_df_copy['week'] = dataset_df_copy['disbursementdate'].dt.to_period('W')
#         weekly_gini = dataset_df_copy.groupby('week').apply(
#             lambda x: calculate_gini(x[score_column], x[label_column])
#             if len(x) >= 10 else np.nan
#         ).reset_index(name='gini')
#         weekly_gini['period'] = 'Week'
#         weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
#         weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
        
#         # Add distinct account count for weekly
#         if account_id_column:
#             weekly_gini['distinct_accounts'] = dataset_df_copy.groupby('week')[account_id_column].nunique().values
        
#         weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period']]
        
#         # Calculate monthly Gini
#         dataset_df_copy = dataset_df.copy()
#         dataset_df_copy['month'] = dataset_df_copy['disbursementdate'].dt.to_period('M')
#         monthly_gini = dataset_df_copy.groupby('month').apply(
#             lambda x: calculate_gini(x[score_column], x[label_column])
#             if len(x) >= 20 else np.nan
#         ).reset_index(name='gini')
#         monthly_gini['period'] = 'Month'
#         monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
#         monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
        
#         # Add distinct account count for monthly
#         if account_id_column:
#             monthly_gini['distinct_accounts'] = dataset_df_copy.groupby('month')[account_id_column].nunique().values
        
#         monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period']]
        
#         # Combine results for this dataset
#         gini_results = pd.concat([weekly_gini, monthly_gini], ignore_index=True)
#         gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
        
#         # Add metadata columns
#         gini_results['Model_Name'] = score_column
#         gini_results['bad_rate'] = namecolumn
#         gini_results['segment_type'] = dataset_name
        
#         # Add individual segment components
#         if model_version_column and model_version_column in metadata:
#             gini_results['model_version'] = metadata[model_version_column]
#         else:
#             gini_results['model_version'] = None
        
#         if trench_column and trench_column in metadata:
#             gini_results['trench_category'] = metadata[trench_column]
#         else:
#             gini_results['trench_category'] = None
        
#         if loan_type_column and loan_type_column in metadata:
#             gini_results['loan_type'] = metadata[loan_type_column]
#         else:
#             gini_results['loan_type'] = None
        
#         if loan_product_type_column and loan_product_type_column in metadata:
#             gini_results['loan_product_type'] = metadata[loan_product_type_column]
#         else:
#             gini_results['loan_product_type'] = None
        
#         gini_results.rename(columns={'gini': f'{score_column}_{namecolumn}_gini'}, inplace=True)
        
#         all_results.append(gini_results)
    
#     # Combine all results
#     final_results = pd.concat(all_results, ignore_index=True)
    
#     return final_results


# # Usage:
# # gini_results = calculate_periodic_gini_prod_ver_trench(
# #     df_concat, 
# #     'Alpha_cic_sil_score', 
# #     'deffpd0', 
# #     'FPD0',
# #     model_version_column='modelVersionId',
# #     trench_column='trenchCategory',
# #     loan_type_column='loan_type',
# #     loan_product_type_column='loan_product_type',
# #     account_id_column='digitalLoanAccountId'
# # )

In [5]:
import pandas as pd
import numpy as np
from datetime import timedelta
from itertools import combinations

def calculate_gini(scores, labels):
    """Calculate Gini coefficient"""
    n = len(scores)
    if n < 2:
        return np.nan
    
    sorted_indices = np.argsort(scores)
    sorted_labels = labels.iloc[sorted_indices].values
    cumsum_labels = np.cumsum(sorted_labels)
    
    gini = 1 - 2 * np.sum(cumsum_labels) / (n * np.sum(sorted_labels))
    return gini


def calculate_periodic_gini_prod_ver_trench_md(df, score_column, label_column, namecolumn, 
                                        model_version_column=None, trench_column=None, 
                                        loan_type_column=None, loan_product_type_column=None,
                                        account_id_column=None):
    """
    Calculate periodic Gini coefficients at multiple levels:
    - Overall (entire dataset)
    - By each individual segment (model version, trench, loan type, loan product type)
    - By all possible combinations of segments
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    namecolumn: name for the bad rate label
    model_version_column: (optional) name of column for model version
    trench_column: (optional) name of column for trench category
    loan_type_column: (optional) name of loan type column
    loan_product_type_column: (optional) name of loan product type column
    account_id_column: (optional) name of column for distinct account IDs
    """
    # Input validation
    required_columns = ['disbursementdate', score_column, label_column]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
    
    optional_columns = {
        'model_version': model_version_column,
        'trench': trench_column,
        'loan_type': loan_type_column,
        'loan_product_type': loan_product_type_column,
        'account_id': account_id_column
    }
    
    for col_name, col in optional_columns.items():
        if col and col not in df.columns:
            raise ValueError(f"{col_name.replace('_', ' ').title()} column '{col}' not found in dataframe")
    
    # Create a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Ensure score and label columns are numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
    # Drop rows with invalid values
    df = df.dropna(subset=[score_column, label_column])
    
    # Define list of datasets to process
    datasets_to_process = [('Overall', df, {})]
    
    # Create list of available segment columns
    segment_columns = []
    if model_version_column:
        segment_columns.append(('ModelVersion', model_version_column))
    if trench_column:
        segment_columns.append(('Trench', trench_column))
    if loan_type_column:
        segment_columns.append(('LoanType', loan_type_column))
    if loan_product_type_column:
        segment_columns.append(('ProductType', loan_product_type_column))
    
    # Generate all possible combinations of segment columns (from single to all)
    for r in range(1, len(segment_columns) + 1):
        for combo in combinations(segment_columns, r):
            # Create combinations by iterating through all segment dimensions
            def generate_combinations(df, segment_columns, index=0, current_filter=None, current_name=''):
                if current_filter is None:
                    current_filter = {}
                
                if index >= len(segment_columns):
                    # Apply all filters and create segment
                    filtered_df = df
                    for col, val in current_filter.items():
                        filtered_df = filtered_df[filtered_df[col] == val]
                    
                    if len(filtered_df) > 0:
                        yield (current_name.strip('_'), filtered_df, current_filter.copy())
                    return
                
                seg_name, seg_col = segment_columns[index]
                for seg_value in sorted(df[seg_col].dropna().unique()):
                    new_filter = current_filter.copy()
                    new_filter[seg_col] = seg_value
                    new_name = current_name + f'{seg_name}_{seg_value}_'
                    
                    yield from generate_combinations(df, segment_columns, index + 1, new_filter, new_name)
            
            for combo_name, combo_df, combo_metadata in generate_combinations(df, list(combo)):
                datasets_to_process.append((combo_name, combo_df, combo_metadata))
    
    all_results = []
    
    # Process each dataset
    for dataset_name, dataset_df, metadata in datasets_to_process:
        # Calculate weekly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['week'] = dataset_df_copy['disbursementdate'].dt.to_period('W')
        weekly_gini = dataset_df_copy.groupby('week').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 10 else np.nan
        ).reset_index(name='gini')
        weekly_gini['period'] = 'Week'
        weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
        weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
        
        # Add distinct account count for weekly
        if account_id_column:
            weekly_account_counts = dataset_df_copy.groupby('week')[account_id_column].nunique().reset_index()
            weekly_account_counts.columns = ['week', 'distinct_accounts']
            weekly_gini = weekly_gini.merge(weekly_account_counts, on='week', how='left')
        
        weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period'] + (['distinct_accounts'] if account_id_column else [])]
        
        # Calculate monthly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['month'] = dataset_df_copy['disbursementdate'].dt.to_period('M')
        monthly_gini = dataset_df_copy.groupby('month').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 20 else np.nan
        ).reset_index(name='gini')
        monthly_gini['period'] = 'Month'
        monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
        monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
        
        # Add distinct account count for monthly
        if account_id_column:
            monthly_account_counts = dataset_df_copy.groupby('month')[account_id_column].nunique().reset_index()
            monthly_account_counts.columns = ['month', 'distinct_accounts']
            monthly_gini = monthly_gini.merge(monthly_account_counts, on='month', how='left')
        
        monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period'] + (['distinct_accounts'] if account_id_column else [])]
        
        # Combine results for this dataset
        gini_results = pd.concat([weekly_gini, monthly_gini], ignore_index=True)
        gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
        
        # Add metadata columns
        gini_results['Model_Name'] = score_column
        gini_results['bad_rate'] = namecolumn
        gini_results['segment_type'] = dataset_name
        
        # Add individual segment components
        if model_version_column and model_version_column in metadata:
            gini_results['model_version'] = metadata[model_version_column]
        else:
            gini_results['model_version'] = None
        
        if trench_column and trench_column in metadata:
            gini_results['trench_category'] = metadata[trench_column]
        else:
            gini_results['trench_category'] = None
        
        if loan_type_column and loan_type_column in metadata:
            gini_results['loan_type'] = metadata[loan_type_column]
        else:
            gini_results['loan_type'] = None
        
        if loan_product_type_column and loan_product_type_column in metadata:
            gini_results['loan_product_type'] = metadata[loan_product_type_column]
        else:
            gini_results['loan_product_type'] = None
        
        gini_results.rename(columns={'gini': f'{score_column}_{namecolumn}_gini'}, inplace=True)
        
        all_results.append(gini_results)
    
    # Combine all results
    final_results = pd.concat(all_results, ignore_index=True)
    
    return final_results


# Usage:
# gini_results = calculate_periodic_gini_prod_ver_trench_md(
#     df_concat, 
#     'Alpha_cic_sil_score', 
#     'deffpd0', 
#     'FPD0',
#     model_version_column='modelVersionId',
#     trench_column='trenchCategory',
#     loan_type_column='loan_type',
#     loan_product_type_column='loan_product_type',
#     account_id_column='digitalLoanAccountId'
# )

In [6]:
import pandas as pd
import numpy as np
from datetime import timedelta
from itertools import combinations

def calculate_gini(scores, labels):
    """Calculate Gini coefficient"""
    n = len(scores)
    if n < 2:
        return np.nan
    
    sorted_indices = np.argsort(scores)
    sorted_labels = labels.iloc[sorted_indices].values
    cumsum_labels = np.cumsum(sorted_labels)
    
    gini = 1 - 2 * np.sum(cumsum_labels) / (n * np.sum(sorted_labels))
    return gini


def calculate_periodic_gini_prod_ver_trench_md_col(df, score_column, label_column, namecolumn, 
                                        model_version_column=None, trench_column=None, 
                                        loan_type_column=None, loan_product_type_column=None,
                                        account_id_column=None):
    """
    Calculate periodic Gini coefficients at multiple levels (wide format):
    - Overall (entire dataset)
    - By each individual segment (model version, trench, loan type, loan product type)
    - By all possible combinations of segments
    
    Returns wide format with one row per period and separate columns for each segment_type's Gini.
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    namecolumn: name for the bad rate label
    model_version_column: (optional) name of column for model version
    trench_column: (optional) name of column for trench category
    loan_type_column: (optional) name of loan type column
    loan_product_type_column: (optional) name of loan product type column
    account_id_column: (optional) name of column for distinct account IDs
    """
    # Input validation
    required_columns = ['disbursementdate', score_column, label_column]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
    
    optional_columns = {
        'model_version': model_version_column,
        'trench': trench_column,
        'loan_type': loan_type_column,
        'loan_product_type': loan_product_type_column,
        'account_id': account_id_column
    }
    
    for col_name, col in optional_columns.items():
        if col and col not in df.columns:
            raise ValueError(f"{col_name.replace('_', ' ').title()} column '{col}' not found in dataframe")
    
    # Create a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Ensure score and label columns are numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
    # Drop rows with invalid values
    df = df.dropna(subset=[score_column, label_column])
    
    # Define list of datasets to process
    datasets_to_process = [('Overall', df, {})]
    
    # Create list of available segment columns
    segment_columns = []
    if model_version_column:
        segment_columns.append(('ModelVersion', model_version_column))
    if trench_column:
        segment_columns.append(('Trench', trench_column))
    if loan_type_column:
        segment_columns.append(('LoanType', loan_type_column))
    if loan_product_type_column:
        segment_columns.append(('ProductType', loan_product_type_column))
    
    # Generate all possible combinations of segment columns (from single to all)
    for r in range(1, len(segment_columns) + 1):
        for combo in combinations(segment_columns, r):
            # Create combinations by iterating through all segment dimensions
            def generate_combinations(df, segment_columns, index=0, current_filter=None, current_name=''):
                if current_filter is None:
                    current_filter = {}
                
                if index >= len(segment_columns):
                    # Apply all filters and create segment
                    filtered_df = df
                    for col, val in current_filter.items():
                        filtered_df = filtered_df[filtered_df[col] == val]
                    
                    if len(filtered_df) > 0:
                        yield (current_name.strip('_'), filtered_df, current_filter.copy())
                    return
                
                seg_name, seg_col = segment_columns[index]
                for seg_value in sorted(df[seg_col].dropna().unique()):
                    new_filter = current_filter.copy()
                    new_filter[seg_col] = seg_value
                    new_name = current_name + f'{seg_name}_{seg_value}_'
                    
                    yield from generate_combinations(df, segment_columns, index + 1, new_filter, new_name)
            
            for combo_name, combo_df, combo_metadata in generate_combinations(df, list(combo)):
                datasets_to_process.append((combo_name, combo_df, combo_metadata))
    
    all_results = []
    
    # Process each dataset
    for dataset_name, dataset_df, metadata in datasets_to_process:
        # Calculate weekly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['week'] = dataset_df_copy['disbursementdate'].dt.to_period('W')
        weekly_gini = dataset_df_copy.groupby('week').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 10 else np.nan
        ).reset_index(name='gini')
        weekly_gini['period'] = 'Week'
        weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
        weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
        weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period']]
        
        # Calculate monthly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['month'] = dataset_df_copy['disbursementdate'].dt.to_period('M')
        monthly_gini = dataset_df_copy.groupby('month').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 20 else np.nan
        ).reset_index(name='gini')
        monthly_gini['period'] = 'Month'
        monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
        monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
        monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period']]
        
        # Combine results for this dataset
        gini_results = pd.concat([weekly_gini, monthly_gini], ignore_index=True)
        gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
        
        # Create column name for this segment's Gini
        gini_column_name = f'{score_column}_{namecolumn}_gini_{dataset_name}'
        gini_results[gini_column_name] = gini_results['gini']
        gini_results = gini_results.drop('gini', axis=1)
        
        all_results.append(gini_results)
    
    # Merge all results on start_date, end_date, and period
    final_results = all_results[0]
    for result_df in all_results[1:]:
        final_results = final_results.merge(
            result_df,
            on=['start_date', 'end_date', 'period'],
            how='outer'
        )
    
    # Add metadata columns (these will be the same for each row)
    final_results['Model_Name'] = score_column
    final_results['bad_rate'] = namecolumn
    
    if model_version_column:
        final_results['model_version'] = None
    else:
        final_results['model_version'] = None
    
    if trench_column:
        final_results['trench_category'] = None
    else:
        final_results['trench_category'] = None
    
    if loan_type_column:
        final_results['loan_type'] = None
    else:
        final_results['loan_type'] = None
    
    if loan_product_type_column:
        final_results['loan_product_type'] = None
    else:
        final_results['loan_product_type'] = None
    
    # Reorder columns: metadata first, then all Gini columns
    metadata_cols = ['start_date', 'end_date', 'period', 'Model_Name', 'bad_rate', 
                     'model_version', 'trench_category', 'loan_type', 'loan_product_type']
    gini_cols = [col for col in final_results.columns if col.startswith(f'{score_column}_{namecolumn}_gini_')]
    
    final_results = final_results[metadata_cols + gini_cols]
    
    return final_results


# Usage:
# gini_results = calculate_periodic_gini_prod_ver_trench_md(
#     df_concat, 
#     'Alpha_cic_sil_score', 
#     'deffpd0', 
#     'FPD0',
#     model_version_column='modelVersionId',
#     trench_column='trenchCategory',
#     loan_type_column='loan_type',
#     loan_product_type_column='loan_product_type',
#     account_id_column='digitalLoanAccountId'
# )

In [7]:
import pandas as pd
import numpy as np
from datetime import timedelta
from itertools import combinations

def calculate_gini(scores, labels):
    """Calculate Gini coefficient"""
    n = len(scores)
    if n < 2:
        return np.nan
    
    sorted_indices = np.argsort(scores)
    sorted_labels = labels.iloc[sorted_indices].values
    cumsum_labels = np.cumsum(sorted_labels)
    
    gini = 1 - 2 * np.sum(cumsum_labels) / (n * np.sum(sorted_labels))
    return gini


def calculate_periodic_gini_prod_ver_trench_dimfact(df, score_column, label_column, namecolumn, 
                                        model_version_column=None, trench_column=None, 
                                        loan_type_column=None, loan_product_type_column=None,
                                        account_id_column=None):
    """
    Calculate periodic Gini coefficients and return Power BI-friendly long format
    with fact and dimension tables.
    
    Returns:
    - fact_table: Long format with one row per segment per period
    - dimension_table: Unique segment combinations for filtering
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    namecolumn: name for the bad rate label
    model_version_column: (optional) name of column for model version
    trench_column: (optional) name of column for trench category
    loan_type_column: (optional) name of loan type column
    loan_product_type_column: (optional) name of loan product type column
    account_id_column: (optional) name of column for distinct account IDs
    """
    # Input validation
    required_columns = ['disbursementdate', score_column, label_column]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
    
    optional_columns = {
        'model_version': model_version_column,
        'trench': trench_column,
        'loan_type': loan_type_column,
        'loan_product_type': loan_product_type_column,
        'account_id': account_id_column
    }
    
    for col_name, col in optional_columns.items():
        if col and col not in df.columns:
            raise ValueError(f"{col_name.replace('_', ' ').title()} column '{col}' not found in dataframe")
    
    # Create a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Ensure score and label columns are numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
    # Drop rows with invalid values
    df = df.dropna(subset=[score_column, label_column])
    
    # Define list of datasets to process
    datasets_to_process = [('Overall', df, {})]
    
    # Create list of available segment columns
    segment_columns = []
    if model_version_column:
        segment_columns.append(('ModelVersion', model_version_column))
    if trench_column:
        segment_columns.append(('Trench', trench_column))
    if loan_type_column:
        segment_columns.append(('LoanType', loan_type_column))
    if loan_product_type_column:
        segment_columns.append(('ProductType', loan_product_type_column))
    
    # Generate all possible combinations of segment columns
    for r in range(1, len(segment_columns) + 1):
        for combo in combinations(segment_columns, r):
            def generate_combinations(df, segment_columns, index=0, current_filter=None, current_name=''):
                if current_filter is None:
                    current_filter = {}
                
                if index >= len(segment_columns):
                    filtered_df = df
                    for col, val in current_filter.items():
                        filtered_df = filtered_df[filtered_df[col] == val]
                    
                    if len(filtered_df) > 0:
                        yield (current_name.strip('_'), filtered_df, current_filter.copy())
                    return
                
                seg_name, seg_col = segment_columns[index]
                for seg_value in sorted(df[seg_col].dropna().unique()):
                    new_filter = current_filter.copy()
                    new_filter[seg_col] = seg_value
                    new_name = current_name + f'{seg_name}_{seg_value}_'
                    
                    yield from generate_combinations(df, segment_columns, index + 1, new_filter, new_name)
            
            for combo_name, combo_df, combo_metadata in generate_combinations(df, list(combo)):
                datasets_to_process.append((combo_name, combo_df, combo_metadata))
    
    all_results = []
    
    # Process each dataset
    for dataset_name, dataset_df, metadata in datasets_to_process:
        # Calculate weekly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['week'] = dataset_df_copy['disbursementdate'].dt.to_period('W')
        weekly_gini = dataset_df_copy.groupby('week').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 10 else np.nan
        ).reset_index(name='gini_value')
        weekly_gini['period'] = 'Week'
        weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
        weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
        
        # Add distinct account count for weekly
        if account_id_column:
            weekly_account_counts = dataset_df_copy.groupby('week')[account_id_column].nunique().reset_index()
            weekly_account_counts.columns = ['week', 'distinct_accounts']
            weekly_gini = weekly_gini.merge(weekly_account_counts, on='week', how='left')
        else:
            weekly_gini['distinct_accounts'] = None
        
        weekly_gini = weekly_gini[['start_date', 'end_date', 'gini_value', 'period', 'distinct_accounts']]
        
        # Calculate monthly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['month'] = dataset_df_copy['disbursementdate'].dt.to_period('M')
        monthly_gini = dataset_df_copy.groupby('month').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 20 else np.nan
        ).reset_index(name='gini_value')
        monthly_gini['period'] = 'Month'
        monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
        monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
        
        # Add distinct account count for monthly
        if account_id_column:
            monthly_account_counts = dataset_df_copy.groupby('month')[account_id_column].nunique().reset_index()
            monthly_account_counts.columns = ['month', 'distinct_accounts']
            monthly_gini = monthly_gini.merge(monthly_account_counts, on='month', how='left')
        else:
            monthly_gini['distinct_accounts'] = None
        
        monthly_gini = monthly_gini[['start_date', 'end_date', 'gini_value', 'period', 'distinct_accounts']]
        
        # Combine results for this dataset
        gini_results = pd.concat([weekly_gini, monthly_gini], ignore_index=True)
        gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
        
        # Add metadata columns
        gini_results['Model_Name'] = score_column
        gini_results['bad_rate'] = namecolumn
        gini_results['segment_type'] = dataset_name
        
        # Add individual segment components
        gini_results['model_version'] = metadata.get(model_version_column, None) if model_version_column else None
        gini_results['trench_category'] = metadata.get(trench_column, None) if trench_column else None
        gini_results['loan_type'] = metadata.get(loan_type_column, None) if loan_type_column else None
        gini_results['loan_product_type'] = metadata.get(loan_product_type_column, None) if loan_product_type_column else None
        
        all_results.append(gini_results)
    
    # Combine all results
    fact_table = pd.concat(all_results, ignore_index=True)
    
    # Create dimension table (unique segment combinations for filtering)
    dimension_table = fact_table[['Model_Name', 'bad_rate', 'segment_type', 'model_version', 
                                   'trench_category', 'loan_type', 'loan_product_type']].drop_duplicates().reset_index(drop=True)
    dimension_table['segment_id'] = range(len(dimension_table))
    
    # Add segment_id to fact table
    fact_table = fact_table.merge(dimension_table[['segment_id', 'Model_Name', 'bad_rate', 'segment_type', 
                                                     'model_version', 'trench_category', 'loan_type', 'loan_product_type']], 
                                  on=['Model_Name', 'bad_rate', 'segment_type', 'model_version', 
                                      'trench_category', 'loan_type', 'loan_product_type'], 
                                  how='left')
    
    # Reorder columns in fact table
    fact_table = fact_table[['segment_id', 'start_date', 'end_date', 'period', 'gini_value', 'distinct_accounts',
                             'Model_Name', 'bad_rate', 'segment_type', 'model_version', 'trench_category', 
                             'loan_type', 'loan_product_type']]
    
    return fact_table, dimension_table


# Usage:
# fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_md(
#     df_concat, 
#     'Alpha_cic_sil_score', 
#     'deffpd0', 
#     'FPD0',
#     model_version_column='modelVersionId',
#     trench_column='trenchCategory',
#     loan_type_column='loan_type',
#     loan_product_type_column='loan_product_type',
#     account_id_column='digitalLoanAccountId'
# )
# 
# # In Power BI:
# # 1. Import fact_table and dimension_table
# # 2. Create relationship: dimension_table[segment_id] -> fact_table[segment_id]
# # 3. Use dimension table columns as filters
# # 4. Create DAX measures:
# #    - Gini Measure = AVERAGE(fact_table[gini_value])
# #    - Account Count = SUM(fact_table[distinct_accounts])
# # 5. Use start_date, end_date, period for time-based analysis

# SIL

## FPD0

### Test

In [8]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId, 
    case when trenchCategory is null then 'ALL'
         when trenchCategory = '' then 'ALL'
         else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.audit_balance.ml_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil', 'Sil-Alpha-CIC-SIL-Model')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fpd0 = 1
  )
  select *  from base
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()



The shape of the dataframe downloaded is:	 (48198, 15)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,modelVersionId,trenchCategory,loan_product_type
0,3676303,044508eb-5e12-438c-b1ef-86a21ced7d1e,60836763030011,cic_model_sil,0.04965601193365858,2025-09-10 16:41:44,2025-09-10,2025-09,Test,0,1,SIL-Instore,v1,ALL,Mall
1,3676345,0607f9c9-e3ba-49e4-9485-1acfa99ae525,60836763450011,cic_model_sil,0.14804231980497298,2025-09-10 16:23:25,2025-09-10,2025-09,Test,0,1,SIL Competitor,v1,ALL,Appliance
2,3676152,07edc892-0d41-439e-ad55-f8165c0427cc,60836761520013,cic_model_sil,0.07974516024873278,2025-09-10 15:36:01,2025-09-10,2025-09,Test,0,1,SIL Competitor,v1,ALL,Appliance
3,3676262,4281cbb6-49de-429f-9c53-f4344003c6b7,60836762620018,cic_model_sil,0.11136085020144332,2025-09-10 15:58:54,2025-09-10,2025-09,Test,0,1,SIL Competitor,v1,ALL,Appliance
4,3676302,4e405e33-5b05-4955-97e5-10722dbf2fd3,60836763020019,cic_model_sil,0.11104678305795064,2025-09-10 16:12:22,2025-09-10,2025-09,Test,0,1,SIL-Instore,v1,ALL,Appliance


In [9]:
df1 = dfd.copy()

### Train

In [10]:
sq = """ 
  with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId, 
    case when trenchCategory is null then 'ALL' 
        when trenchCategory = ''then 'ALL' 
        else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil', 'Sil-Alpha-CIC-SIL-Model')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as
(select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
    deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
  modelVersionId,
    trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
    from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fpd0 = 1
  )
  select * from base;
  """
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

The shape of the dataframe downloaded is:	 (292181, 15)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,modelVersionId,trenchCategory,loan_product_type
0,2394977,b74071a9-2f14-4e55-9bc5-cbfcb7344c5a,60823949770013,Alpha - CIC-SIL-Model,0.108317,2024-02-09 16:42:24,2024-02-09,2024-02,Train,0,1,SIL-Instore,v1,ALL,Appliance
1,2393585,af1763b6-3754-4b46-9ccc-4a3908ca68cb,60823935850017,Alpha - CIC-SIL-Model,0.117100,2024-02-08 12:18:12,2024-02-08,2024-02,Train,1,1,SIL-Instore,v1,ALL,Appliance
2,2117667,e7a1ab7a-242b-41bf-8845-c5a0a257515a,60821176670012,Alpha - CIC-SIL-Model,0.165660,2023-10-27 20:07:24,2023-10-27,2023-10,Train,0,1,SIL-Instore,v1,ALL,Appliance
3,2254093,e0a87777-f74b-4706-a3e3-51d461366b13,60822540930011,Alpha - CIC-SIL-Model,0.161881,2023-09-30 17:24:32,2023-09-30,2023-09,Train,0,1,SIL-Instore,v1,ALL,Appliance
4,2388357,81a58864-e04c-4055-a4b6-65e361c06f0e,60823883570012,Alpha - CIC-SIL-Model,0.125133,2024-02-03 11:01:47,2024-02-03,2024-02,Train,1,1,SIL-Instore,v1,ALL,Appliance


In [11]:
df2 = dfd.copy()

### Concat

In [12]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()

The shape of the concatenated dataframe is:	 (340379, 15)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,Alpha_cic_sil_score,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,modelVersionId,trenchCategory,loan_product_type
0,3676303,044508eb-5e12-438c-b1ef-86a21ced7d1e,60836763030011,cic_model_sil,0.04965601193365858,2025-09-10 16:41:44,2025-09-10,2025-09,Test,0,1,SIL-Instore,v1,ALL,Mall
1,3676345,0607f9c9-e3ba-49e4-9485-1acfa99ae525,60836763450011,cic_model_sil,0.14804231980497298,2025-09-10 16:23:25,2025-09-10,2025-09,Test,0,1,SIL Competitor,v1,ALL,Appliance
2,3676152,07edc892-0d41-439e-ad55-f8165c0427cc,60836761520013,cic_model_sil,0.07974516024873278,2025-09-10 15:36:01,2025-09-10,2025-09,Test,0,1,SIL Competitor,v1,ALL,Appliance
3,3676262,4281cbb6-49de-429f-9c53-f4344003c6b7,60836762620018,cic_model_sil,0.11136085020144332,2025-09-10 15:58:54,2025-09-10,2025-09,Test,0,1,SIL Competitor,v1,ALL,Appliance
4,3676302,4e405e33-5b05-4955-97e5-10722dbf2fd3,60836763020019,cic_model_sil,0.11104678305795064,2025-09-10 16:12:22,2025-09-10,2025-09,Test,0,1,SIL-Instore,v1,ALL,Appliance


In [13]:
df_concat.columns

Index(['customerId', 'digitalLoanAccountId', 'loanAccountNumber',
       'modelDisplayName', 'Alpha_cic_sil_score', 'appln_submit_datetime',
       'disbursementdate', 'Application_month', 'Data_selection', 'deffpd0',
       'flg_mature_fpd0', 'new_loan_type', 'modelVersionId', 'trenchCategory',
       'loan_product_type'],
      dtype='object')

In [14]:
df_concat.groupby(['Data_selection','new_loan_type', 'modelVersionId', 'loan_product_type'])['digitalLoanAccountId'].nunique()

Data_selection  new_loan_type   modelVersionId  loan_product_type
Test            SIL Competitor  v1              Appliance            14107
                                                Mall                  4679
                SIL ZERO        v1              Appliance             2005
                SIL-Instore     v1              Appliance            22617
                                                Mall                  4790
Train           SIL Competitor  v1              Appliance             4595
                                                Mall                  1104
                                                Mobile                  45
                                v2              Appliance            27940
                                                Mall                  8153
                                                Mobile                  45
                SIL Repeat      v1              Appliance             1224
                                  

In [15]:
df_concat['new_loan_type'].unique()

array(['SIL-Instore', 'SIL Competitor', 'SIL ZERO', 'SIL Repeat'],
      dtype=object)

In [16]:
df_concat.to_csv(r"D:\OneDrive - Tonik Financial Pte Ltd\MyStuff\Data Engineering\Model_Monitoring\Gini Monitoring\New_Gini_Monitoring_2025-11-25\Future\sample.csv", index = False)

In [ ]:
fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
    df_concat, 
    'Alpha_cic_sil_score', 
    'deffpd0', 
    'FPD0',
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
    account_id_column='digitalLoanAccountId'
)

In [ ]:
print(fact_table.columns)
fact_table['segment_type'].value_counts(dropna=False)
fact_table['Model_display_name'] = 'cic_model_sil'

In [ ]:
fact_table[fact_table['loan_product_type'].isnull()].head(10)

In [ ]:
dimension_table['Model_display_name'] = 'cic_model_sil'

In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.fact_table"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(fact_table, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.dimension_table"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dimension_table, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

## FPD10

### Test

In [ ]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId, 
    case when trenchCategory is null then 'ALL'
         when trenchCategory = '' then 'ALL'
         else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.audit_balance.ml_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil', 'Sil-Alpha-CIC-SIL-Model')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  deffpd10,
  flg_mature_fpd10,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fpd10 = 1
  )
  select *  from base
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()



In [ ]:
df1 = dfd.copy()

### Train

In [ ]:
sq = """ 
  with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId, 
    case when trenchCategory is null then 'ALL' 
        when trenchCategory = ''then 'ALL' 
        else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil', 'Sil-Alpha-CIC-SIL-Model')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as
(select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
    del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  modelVersionId,
    trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
    from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and del.flg_mature_fpd10 = 1
  )
  select * from base;
  """
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

In [ ]:
df2 = dfd.copy()

### Concat

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()

In [ ]:
# df_concat.to_csv("sample.csv")

In [ ]:
df_concat.groupby(['Data_selection','new_loan_type', 'modelVersionId', 'loan_product_type'])['digitalLoanAccountId'].nunique()

In [ ]:
gini_results = calculate_periodic_gini_prod_ver_trench_md_col(
    df_concat, 
    'Alpha_cic_sil_score', 
    'deffpd10', 
    'FPD10',
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
    account_id_column='digitalLoanAccountId'
)

In [ ]:
print(f"The shape of the Gini results dataframe is:\t {gini_results.shape}")
gini_results.head()

In [ ]:
f1 = gini_results.copy()


## FPD30

### Test

In [ ]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId, 
    case when trenchCategory is null then 'ALL'
         when trenchCategory = '' then 'ALL'
         else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.audit_balance.ml_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil', 'Sil-Alpha-CIC-SIL-Model')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and flg_mature_fpd30 = 1
  )
  select *  from base
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()



In [ ]:
df1 = dfd.copy()

### Train

In [ ]:
sq = """ 
  with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId, 
    case when trenchCategory is null then 'ALL' 
        when trenchCategory = ''then 'ALL' 
        else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil', 'Sil-Alpha-CIC-SIL-Model')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as
(select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
    del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  modelVersionId,
    trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
    from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and del.flg_mature_fpd30 = 1
  )
  select * from base;
  """
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

In [ ]:
df2 = dfd.copy()

### Concat

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()

In [ ]:
# df_concat.to_csv("sample.csv")

In [ ]:
df_concat.groupby(['Data_selection','new_loan_type', 'modelVersionId', 'loan_product_type'])['digitalLoanAccountId'].nunique()

In [ ]:
gini_results = calculate_periodic_gini_prod_ver_trench_md_col(
    df_concat, 
    'Alpha_cic_sil_score', 
    'deffpd30', 
    'FPD30',
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
    account_id_column='digitalLoanAccountId'
)

In [ ]:
print(f"The shape of the Gini results dataframe is:\t {gini_results.shape}")
gini_results.head()

In [ ]:
f2 = gini_results.copy()


## FSPD30

### Test

In [ ]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId, 
    case when trenchCategory is null then 'ALL'
         when trenchCategory = '' then 'ALL'
         else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.audit_balance.ml_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil', 'Sil-Alpha-CIC-SIL-Model')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and del.flg_mature_fspd_30 = 1
  )
  select *  from base
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()



In [ ]:
df1 = dfd.copy()

### Train

In [ ]:
sq = """ 
  with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId, 
    case when trenchCategory is null then 'ALL' 
        when trenchCategory = ''then 'ALL' 
        else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil', 'Sil-Alpha-CIC-SIL-Model')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as
(select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
    del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  modelVersionId,
    trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
    from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and del.flg_mature_fspd_30 = 1
  )
  select * from base;
  """
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

In [ ]:
df2 = dfd.copy()

### Concat

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()

In [ ]:
# df_concat.to_csv("sample.csv")

In [ ]:
df_concat.groupby(['Data_selection','new_loan_type', 'modelVersionId', 'loan_product_type'])['digitalLoanAccountId'].nunique()

In [ ]:
gini_results = calculate_periodic_gini_prod_ver_trench_md_col(
    df_concat, 
    'Alpha_cic_sil_score', 
    'deffspd30', 
    'FSPD30',
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
    account_id_column='digitalLoanAccountId'
)

In [ ]:
print(f"The shape of the Gini results dataframe is:\t {gini_results.shape}")
gini_results.head()

In [ ]:
f3 = gini_results.copy()
f3.head()

## FSTPD30

### Test

In [ ]:
sq = """ 
with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId, 
    case when trenchCategory is null then 'ALL'
         when trenchCategory = '' then 'ALL'
         else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.audit_balance.ml_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil', 'Sil-Alpha-CIC-SIL-Model')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Test' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and del.flg_mature_fstpd_30 = 1
  )
  select *  from base
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()



In [ ]:
df1 = dfd.copy()

### Train

In [ ]:
sq = """ 
  with modelname as 
  (SELECT
    customerId,digitalLoanAccountId,prediction Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId, 
    case when trenchCategory is null then 'ALL' 
        when trenchCategory = ''then 'ALL' 
        else trenchCategory end as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil', 'Sil-Alpha-CIC-SIL-Model')
  ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as
(select r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  r.Alpha_cic_sil_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Train' Data_selection,  
    del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  modelVersionId,
    trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
    from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  inner join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and r.Alpha_cic_sil_score is not null
  and del.flg_mature_fstpd_30 = 1
  )
  select * from base;
  """
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

In [ ]:
df2 = dfd.copy()

### Concat

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()

In [ ]:
# df_concat.to_csv("sample.csv")

In [ ]:
df_concat.groupby(['Data_selection','new_loan_type', 'modelVersionId', 'loan_product_type'])['digitalLoanAccountId'].nunique()

In [ ]:
gini_results = calculate_periodic_gini_prod_ver_trench_md_col(
    df_concat, 
    'Alpha_cic_sil_score', 
    'deffstpd30', 
    'FSTPD30',
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
    account_id_column='digitalLoanAccountId'
)

In [ ]:
print(f"The shape of the Gini results dataframe is:\t {gini_results.shape}")
gini_results.head()

In [ ]:
f4 = gini_results.copy()
f4.head()

## combining the dataframe

In [ ]:
import functools

dataframes = [f0, f1, f2, f3, f4]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name', 'bad_rate', 
       'model_version', 'trench_category', 'loan_type', 'loan_product_type']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

In [ ]:
final_df[['model_version','trench_category']].value_counts(dropna=False)

In [ ]:
# final_df = final_df[['start_date', 'end_date', 
#        'period', 'distinct_accounts', 'Model_Name', 'bad_rate',
#        'segment_type', 'model_version', 'trench_category', 'loan_type',
#        'loan_product_type', 
#        'Alpha_cic_sil_score_FPD0_gini',
#        'Alpha_cic_sil_score_FPD10_gini',
#        'Alpha_cic_sil_score_FPD30_gini',
#        'Alpha_cic_sil_score_FSPD30_gini',
#        'Alpha_cic_sil_score_FSTPD30_gini']].copy()
final_df['Model_display_name'] = 'cic_model_sil'

In [ ]:
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.cic_sil_score_gini_v2"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(final_df, table_id, job_config=job_config)
job.result()  # Wait for the job to complete

In [ ]:
final_df['trench_category'].value_counts(dropna=False)

# End

Not persude as it is not working in power bi